# __Prophet 2019__

In [1]:
!pip install pystan==2.19.1.1

!pip install prophet

     |████████████████████████████████| 16.2 MB 2.4 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.24-cp39-cp39-macosx_10_9_x86_64.whl (1.9 MB)
Using legacy 'setup.py install' for pystan, since package 'wheel' is not installed.
    Running setup.py install for pystan ... done
     |████████████████████████████████| 65 kB 2.1 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 49 kB 12.2 MB/s            
     |████████████████████████████████| 47 kB 6.6 MB/s             
     |████████████████████████████████| 155 kB 9.9 MB/s            
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 45 kB 8.3 MB/s             
     |████████████████████████████████| 5.4 MB 26.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.4 MB 32.0 MB/s            
Using legacy 'setup.py install' for prophet, since package 'wheel' is

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
%matplotlib inline

from prophet import Prophet

import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.seasonal import seasonal_decompose

Importing plotly failed. Interactive plots will not work.


In [4]:
imb2019=pd.read_csv("../data/imbalance_1h_2019.csv")

In [5]:
imb2019.head()

,timestamp,imbalance_price
0,2019-01-01 00:00:00,-329.6045
1,2019-01-01 01:00:00,-283.8000
2,2019-01-01 02:00:00,-334.7170
3,2019-01-01 03:00:00,117.6665
4,2019-01-01 04:00:00,-323.3150


In [7]:
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

def limit(data, frm, to):
    return data[(data.index>=frm)&(data.index<to)]

In [8]:
dataset_hourly = pd.read_csv('../data/imbalance_1h_2019.csv', 
                            index_col=[0], parse_dates=[0])
dataset_hourly.sort_index(inplace=True)

In [10]:
t = dataset_hourly.imbalance_price.copy()
t = t.drop(t.index[t.index.duplicated()])
freq_index = pd.date_range(start=t.index[0], end=t.index[-1], freq='H')
constructed = pd.Series(index=freq_index, name='imbalance_price')
constructed.update(t)
constructed.interpolate(inplace=True)

train, test = split_data(constructed, '01-01-2019')

In [13]:
dataset_hourly.head()

,imbalance_price
timestamp,
2019-01-01 00:00:00,-329.6045
2019-01-01 01:00:00,-283.8000
2019-01-01 02:00:00,-334.7170
2019-01-01 03:00:00,117.6665
2019-01-01 04:00:00,-323.3150


In [14]:
train.head()

2019-01-01 00:00:00   -329.6045
2019-01-01 01:00:00   -283.8000
2019-01-01 02:00:00   -334.7170
2019-01-01 03:00:00    117.6665
2019-01-01 04:00:00   -323.3150
Freq: H, Name: imbalance_price, dtype: float64

In [26]:
train.reset_index()

,index,imbalance_price
0,2019-01-01 00:00:00,-329.6045
1,2019-01-01 01:00:00,-283.8000
2,2019-01-01 02:00:00,-334.7170
3,2019-01-01 03:00:00,117.6665
4,2019-01-01 04:00:00,-323.3150
...,...,...
5803,2019-08-30 19:00:00,-438.8510
5804,2019-08-30 20:00:00,-154.7230
5805,2019-08-30 21:00:00,-12.2765
5806,2019-08-30 22:00:00,262.0530


In [15]:
test.head()

2019-01-09 00:00:00    1112.5615
2019-01-09 01:00:00     842.9860
2019-01-09 02:00:00     624.5080
2019-01-09 03:00:00     827.4690
2019-01-09 04:00:00     689.3620
Freq: H, Name: imbalance_price, dtype: float64

In [ ]:
dataset2_hourly= dataset_hourly.copy()

In [21]:
dataset2_hourly= dataset_hourly.copy()

In [30]:
dataset2_hourly['ds'] = pd.to_datetime(train[1])
#dataset2_hourly = dataset2_hourly[dataset2_hourly['ds'].dt.hour < 1]

In [34]:
train[:]

2019-01-01 00:00:00   -329.6045
2019-01-01 01:00:00   -283.8000
2019-01-01 02:00:00   -334.7170
2019-01-01 03:00:00    117.6665
2019-01-01 04:00:00   -323.3150
                         ...   
2019-08-30 19:00:00   -438.8510
2019-08-30 20:00:00   -154.7230
2019-08-30 21:00:00    -12.2765
2019-08-30 22:00:00    262.0530
2019-08-30 23:00:00    300.0955
Freq: H, Name: imbalance_price, Length: 5808, dtype: float64

In [31]:
dataset2_hourly.head()

,imbalance_price,ds
timestamp,,
2019-01-01 00:00:00,-329.6045,1969-12-31 23:59:59.999999717
2019-01-01 01:00:00,-283.8000,1969-12-31 23:59:59.999999717
2019-01-01 02:00:00,-334.7170,1969-12-31 23:59:59.999999717
2019-01-01 03:00:00,117.6665,1969-12-31 23:59:59.999999717
2019-01-01 04:00:00,-323.3150,1969-12-31 23:59:59.999999717


In [ ]:
m = Prophet().fit(dataset2_hourly)
future = m.make_future_dataframe(periods=300, freq='H')
fcst = m.predict(future)
fig = m.plot(fcst)